In [1]:
import requests
import re

import numpy as np
import pandas as pd
import py_stringsimjoin as ssj
import py_stringmatching as sm

# Collect FDA drug names and schedules

In [2]:
# SparQL query on dbpedia to extract entities (drugs) that have the property "legalUS"
url = 'https://dbpedia.org/sparql/'
query = '''
    SELECT ?drug_name, ?sched
    WHERE {
        ?drug dbp:legalUs ?sched .
         ?drug rdfs:label ?drug_name
         FILTER (LANG(?drug_name) = "en")
}
 '''
r = requests.get(url, params={'format': 'json', 'query': query})
data = r.json()

In [3]:
data

{'head': {'link': [], 'vars': ['drug_name', 'sched']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'drug_name': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Botulinum toxin'},
    'sched': {'type': 'typed-literal',
     'datatype': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString',
     'value': 'Rx-only'}},
   {'drug_name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Clozapine'},
    'sched': {'type': 'typed-literal',
     'datatype': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString',
     'value': 'Rx-only'}},
   {'drug_name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Coca'},
    'sched': {'type': 'typed-literal',
     'datatype': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString',
     'value': 'Schedule II'}},
   {'drug_name': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Hyoscine hydrobromide'},
    'sched': {'type': 'typed-literal',
     'datatype': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString',


In [4]:
# Extract data to dict
results = {}
for item in data['results']['bindings']:
    drug_name = item['drug_name']['value'].upper()
    schedule = item['sched']['value'].upper().replace('℞', 'RX')
    if drug_name not in results:
        results[drug_name] = schedule

In [5]:
# Read data to DataFrame
fda_df = pd.DataFrame.from_dict(results, orient='index', columns=['FDA_SCHEDULE'])
fda_df = fda_df.reset_index().rename(columns={'index': 'GENERIC_NAME'})

In [6]:
fda_df['FDA_SCHEDULE'].unique()

array(['RX-ONLY', 'SCHEDULE II', 'SCHEDULE I', 'UNSCHEDULED',
       'SCHEDULE III', '18', 'OTC', 'SCHEDULE IV', 'RX ONLY',
       'SUPPLEMENT', 'DISCONTINUED', 'LIST I OTC', 'WITHDRAWN',
       'OTC , RX-ONLY', 'OTC SUPPLEMENT',
       'HTTP://DBPEDIA.ORG/RESOURCE/OVER-THE-COUNTER_DRUG',
       'NOT FDA APPROVED; UNSCHEDULED', 'NOT FDA APPROVED',
       'NOT APPROVED', 'UNCONTROLLED',
       'HTTP://DBPEDIA.ORG/RESOURCE/LIST_OF_SCHEDULE_III_DRUGS_(US)', 'S',
       'AVAILABLE ONLY AS LUMACAFTOR/IVACAFTOR WHICH IS RX-ONLY',
       'OTC FOR TOPICAL ADMINISTRATION; RX-ONLY FOR ORAL TABLETS, RECTAL USE AND INTRAVENOUS THERAPY',
       'RX', 'NOT PERMITTED AS DRUG OR SUPPLEMENT',
       'PRESCRIPTION ONLY/OTC', 'PROHIBITED', 'SCHEDULE II, SCHEDULE IV',
       'POM', 'SCHEDULE V', 'NOT SCHEDULED', 'RX-ONLY, OTC',
       'INVESTIGATIONAL', 'UNSCHEDULED; NOT FDA APPROVED',
       'SPECIFICALLY UNSCHEDULED', 'ANALOGUE OF MDMA',
       'BEHIND THE COUNTER ; RX ONLY IN OREGON AND MISSISSIPPI',
 

In [7]:
# Function to clean schedule names
def clean_schedule(x):
    sched = x.strip()
    sched = sched.replace('-', '_')
    sched = re.sub(r'OVER[ _]?THE[ _]?COUNTER', 'OTC', sched)
    sched.replace('HTTP://DBPEDIA.ORG/RESOUCE', '')
    if 'RX' in sched:
        sched = 'RX-ONLY'
    return sched

In [8]:
# Clean schedule names
fda_df['FDA_SCHEDULE'] = fda_df.apply(lambda x: clean_schedule(x['FDA_SCHEDULE']), axis=1)

In [9]:
#fda_df.to_csv('./data/cleaned_files/FDA_schedules.csv', index=False)

# Join FDA drug names with drug_names.csv

In [10]:
DRUGS = './data/cleaned_files/drug_names_additional_column.csv'

In [11]:
drugs_df = pd.read_csv(DRUGS)

In [12]:
drugs_df.shape

(2337, 4)

In [13]:
fda_df.shape

(1335, 2)

In [14]:
drugs_df.head()

,name,common_name,name_type,full_common_name
0,Fentanyl,fentanyl,generic_name,fentanyl
1,Apache,fentanyl,street_name,fentanyl
2,Birria,fentanyl,street_name,fentanyl
3,Blonde,fentanyl,street_name,fentanyl
4,Blue Diamond,fentanyl,street_name,fentanyl


In [15]:
fda_df.head()

,GENERIC_NAME,FDA_SCHEDULE
0,BOTULINUM TOXIN,RX-ONLY
1,CLOZAPINE,RX-ONLY
2,COCA,SCHEDULE II
3,HYOSCINE HYDROBROMIDE,RX-ONLY
4,LAUDANUM,SCHEDULE II


In [16]:
# Convert column names and data to uppercase
drugs_df['name'] = drugs_df['name'].str.upper()
drugs_df['common_name'] = drugs_df['common_name'].str.upper()
drugs_df['name_type'] = drugs_df['name_type'].str.upper()
drugs_df['full_common_name'] = drugs_df['full_common_name'].str.upper()
drugs_df = drugs_df.rename(columns={'name': 'NAME', 'common_name':'GENERIC_NAME', 'name_type': 'NAME_TYPE',
                         'full_common_name': 'FULL_GENERIC_NAME'})

In [17]:
# Initialize tokenizer
tok = sm.AlphabeticTokenizer(return_set=True)

In [18]:
drugs_df['id'] = drugs_df.index
fda_df['id'] = fda_df.index

In [19]:
joined = ssj.jaccard_join(fda_df, drugs_df, 'id', 'id', 'GENERIC_NAME', 'GENERIC_NAME', tok, 0.3, 
                 l_out_attrs=['GENERIC_NAME', 'FDA_SCHEDULE'], 
                 r_out_attrs=['NAME', 'GENERIC_NAME', 'NAME_TYPE', 'FULL_GENERIC_NAME'])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [20]:
joined.head()

,_id,l_id,r_id,l_GENERIC_NAME,l_FDA_SCHEDULE,r_NAME,r_GENERIC_NAME,r_NAME_TYPE,r_FULL_GENERIC_NAME,_sim_score
0,0,711,0,FENTANYL,SCHEDULE II,FENTANYL,FENTANYL,GENERIC_NAME,FENTANYL,1.0
1,1,711,1,FENTANYL,SCHEDULE II,APACHE,FENTANYL,STREET_NAME,FENTANYL,1.0
2,2,711,2,FENTANYL,SCHEDULE II,BIRRIA,FENTANYL,STREET_NAME,FENTANYL,1.0
3,3,711,3,FENTANYL,SCHEDULE II,BLONDE,FENTANYL,STREET_NAME,FENTANYL,1.0
4,4,711,4,FENTANYL,SCHEDULE II,BLUE DIAMOND,FENTANYL,STREET_NAME,FENTANYL,1.0


In [21]:
joined.shape

(2421, 10)

In [22]:
drugs_df['GENERIC_NAME'].unique().shape

(62,)

In [23]:
joined['r_GENERIC_NAME'].unique().shape

(42,)

In [24]:
# There were drugs in drug_names.csv that were not matched
# They will be added back later
not_fda = drugs_df[~drugs_df['GENERIC_NAME'].isin(joined['r_GENERIC_NAME'])]
not_fda['GENERIC_NAME'].unique()

array(['U-47700', 'MARIJUANA CONCENTRATES/HASH OIL',
       'SYNTHETIC CANNABINOIDS', 'SYNTHETIC CATHINONES', 'MEPERIDINE',
       'PSILOCYBIN', 'AYAHUASCA', 'GHB', 'KHAT', 'KRATOM', 'PCP',
       'PEYOTE', 'SALVIA', 'AMYL NITRATE', 'ISOBUTYL NITRATE',
       'NITROUS OXIDE', 'PROPOXYPHENE', 'DXM', 'SUDAFED', 'AMOBARBITAL'],
      dtype=object)

In [25]:
# Their Schedules were manually determined via Google search
not_fda['FDA_SCHEDULE'] = 'SCHEDULE I'

/home/jwahl/anaconda3/envs/DSE203/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
# We're not interested in non-Scheduled drugs
fda_sched = fda_df[fda_df['FDA_SCHEDULE'].isin(['SCHEDULE I',
                                                'SCHEDULE II', 
                                                'SCHEDULE III', 
                                                'SCHEDULE IV'])]
missing_drugs = fda_sched[~fda_sched.index.isin(joined['l_id'].unique())].copy()

In [27]:
missing_drugs.head()

,GENERIC_NAME,FDA_SCHEDULE,id
2,COCA,SCHEDULE II,2
4,LAUDANUM,SCHEDULE II,4
9,SODIUM THIOPENTAL,SCHEDULE III,9
27,CATHINE,SCHEDULE IV,27
28,CATHINONE,SCHEDULE I,28


In [28]:
# Fill missing information
missing_drugs['NAME'] = missing_drugs['GENERIC_NAME']
missing_drugs['NAME_TYPE'] = 'GENERIC_NAME'
missing_drugs['FULL_GENERIC_NAME'] = missing_drugs['GENERIC_NAME']

In [29]:
# Subset joined
new_joined = joined[['r_NAME', 'r_GENERIC_NAME', 'r_NAME_TYPE', 'r_FULL_GENERIC_NAME' ,'l_FDA_SCHEDULE']].copy()
new_joined = new_joined.rename(columns={'r_NAME': 'NAME', 'r_GENERIC_NAME': 'GENERIC_NAME',
                                        'r_NAME_TYPE': 'NAME_TYPE', 'r_FULL_GENERIC_NAME': 'FULL_GENERIC_NAME',
                                        'l_FDA_SCHEDULE': 'FDA_SCHEDULE'})
new_joined

,NAME,GENERIC_NAME,NAME_TYPE,FULL_GENERIC_NAME,FDA_SCHEDULE
0,FENTANYL,FENTANYL,GENERIC_NAME,FENTANYL,SCHEDULE II
1,APACHE,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II
2,BIRRIA,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II
3,BLONDE,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II
4,BLUE DIAMOND,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II
...,...,...,...,...,...
2416,WEIGHT GAINERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III
2417,WEIGHT GAINERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III
2418,STEROIDS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III
2419,STEROIDS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III


In [30]:
# Join data from joined
new_drugs_df = drugs_df.merge(new_joined.drop_duplicates(subset=['GENERIC_NAME'])[['GENERIC_NAME','FDA_SCHEDULE']], 
                   on='GENERIC_NAME', how='left')
new_drugs_df = new_drugs_df.drop(columns=['id'])
new_drugs_df

,NAME,GENERIC_NAME,NAME_TYPE,FULL_GENERIC_NAME,FDA_SCHEDULE
0,FENTANYL,FENTANYL,GENERIC_NAME,FENTANYL,SCHEDULE II
1,APACHE,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II
2,BIRRIA,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II
3,BLONDE,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II
4,BLUE DIAMOND,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II
...,...,...,...,...,...
2332,PUMPERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III
2333,ROIDS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III
2334,STACKERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III
2335,WEIGHT GAINERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III


In [31]:
# Add back missing drugs
new_drugs_df = pd.concat([new_drugs_df, missing_drugs.drop(columns=['id'])])
new_drugs_df.shape

(2636, 5)

In [32]:
#new_drugs_df.to_csv('./data/cleaned_files/drug_names_fda.csv', index=False)

# Match Drugs to VA Categories

In [33]:
VA = './data/cleaned_files/VAProduct_Name_Categories.csv'

In [34]:
va_df = pd.read_csv(VA)

In [35]:
va_df.head()

,GENERIC,VA_PRODUCT,VA_CLASS,NF_INDICATOR,ITEM_STATUS,PPS Add Date,PPS Delete Date,VA Category
0,"1,1,1 TRICHLOROETHANE","1,1,1 TRICHLOROETHANE (INHIBITED) SOLN",PH000,N,INACTIVE,10/16/2012,10/1/2008,PHARMACEUTICAL AIDS/REAGENTS
1,CHROMIC CHLORIDE/CUPRIC SULFATE/MANGANESE SULFATE/ZINC SULFATE,4-MIXED TRACE ELEMENTS INJ,TN490,Y,INACTIVE,10/16/2012,11/1/2005,"ELECTROLYTES/MINERALS,COMBINATIONS"
2,ABACAVIR,"ABACAVIR SO4 100MG/5ML SOLN,ORAL",AM800,Y,ACTIVE,10/16/2012,NaN,ANTIVIRALS
3,ABACAVIR,ABACAVIR SO4 300MG TAB,AM800,Y,ACTIVE,10/16/2012,NaN,ANTIVIRALS
4,ABACAVIR,"ABACAVIR SO4 300MG TAB,UD",AM800,Y,ACTIVE,10/16/2012,NaN,ANTIVIRALS


In [36]:
va_df.shape

(29755, 8)

In [37]:
new_drugs_df['id'] = new_drugs_df.reset_index(drop=True).index
va_df['id'] = va_df.index

In [38]:
# Match on generic names and join VA categories
va_joined = ssj.jaccard_join(new_drugs_df, va_df, 'id', 'id', 'GENERIC_NAME', 'GENERIC', tok, 0.3, 
                 l_out_attrs=['NAME', 'GENERIC_NAME', 'NAME_TYPE', 'FULL_GENERIC_NAME', 'FDA_SCHEDULE'], 
                 r_out_attrs=['VA_CLASS', 'VA Category'])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [39]:
va_joined.head()

,_id,l_id,r_id,l_NAME,l_GENERIC_NAME,l_NAME_TYPE,l_FULL_GENERIC_NAME,l_FDA_SCHEDULE,r_VA_CLASS,r_VA Category,_sim_score
0,0,2246,116,PSEUDOEPHEDRINE,PSEUDOEPHEDRINE,GENERIC_NAME,PSEUDOEPHEDRINE,OTC,RE599,"COLD REMEDIES,OTHER",0.5
1,1,2248,116,CHALK,PSEUDOEPHEDRINE,STREET_NAME,PSEUDOEPHEDRINE,OTC,RE599,"COLD REMEDIES,OTHER",0.5
2,2,2249,116,CRANK,PSEUDOEPHEDRINE,STREET_NAME,PSEUDOEPHEDRINE,OTC,RE599,"COLD REMEDIES,OTHER",0.5
3,3,2250,116,METH,PSEUDOEPHEDRINE,STREET_NAME,PSEUDOEPHEDRINE,OTC,RE599,"COLD REMEDIES,OTHER",0.5
4,4,2251,116,SPEED,PSEUDOEPHEDRINE,STREET_NAME,PSEUDOEPHEDRINE,OTC,RE599,"COLD REMEDIES,OTHER",0.5


In [40]:
va_joined.drop_duplicates(subset=['l_id'])

,_id,l_id,r_id,l_NAME,l_GENERIC_NAME,l_NAME_TYPE,l_FULL_GENERIC_NAME,l_FDA_SCHEDULE,r_VA_CLASS,r_VA Category,_sim_score
0,0,2246,116,PSEUDOEPHEDRINE,PSEUDOEPHEDRINE,GENERIC_NAME,PSEUDOEPHEDRINE,OTC,RE599,"COLD REMEDIES,OTHER",0.5
1,1,2248,116,CHALK,PSEUDOEPHEDRINE,STREET_NAME,PSEUDOEPHEDRINE,OTC,RE599,"COLD REMEDIES,OTHER",0.5
2,2,2249,116,CRANK,PSEUDOEPHEDRINE,STREET_NAME,PSEUDOEPHEDRINE,OTC,RE599,"COLD REMEDIES,OTHER",0.5
3,3,2250,116,METH,PSEUDOEPHEDRINE,STREET_NAME,PSEUDOEPHEDRINE,OTC,RE599,"COLD REMEDIES,OTHER",0.5
4,4,2251,116,SPEED,PSEUDOEPHEDRINE,STREET_NAME,PSEUDOEPHEDRINE,OTC,RE599,"COLD REMEDIES,OTHER",0.5
...,...,...,...,...,...,...,...,...,...,...,...
34291,34291,2258,29731,ROCHE,ZOLPIDEM,STREET_NAME,ZOLPIDEM,SCHEDULE IV,CN309,"SEDATIVES/HYPNOTICS,OTHER",1.0
34292,34292,2259,29731,ROOFIES,ZOLPIDEM,STREET_NAME,ZOLPIDEM,SCHEDULE IV,CN309,"SEDATIVES/HYPNOTICS,OTHER",1.0
34293,34293,2260,29731,ROOFINOL,ZOLPIDEM,STREET_NAME,ZOLPIDEM,SCHEDULE IV,CN309,"SEDATIVES/HYPNOTICS,OTHER",1.0
34294,34294,2261,29731,ROPE,ZOLPIDEM,STREET_NAME,ZOLPIDEM,SCHEDULE IV,CN309,"SEDATIVES/HYPNOTICS,OTHER",1.0


In [41]:
# Not all generic names are matched, so we can just join the categories to the original table
# Missing categories can be filled later
joined_drugs_df = new_drugs_df.join(va_joined.drop_duplicates(subset=['l_id'])[['l_id', 'r_VA_CLASS', 'r_VA Category']].set_index('l_id'))
joined_drugs_df.drop_duplicates(subset=['GENERIC_NAME'])

,NAME,GENERIC_NAME,NAME_TYPE,FULL_GENERIC_NAME,FDA_SCHEDULE,id,r_VA_CLASS,r_VA Category
0,FENTANYL,FENTANYL,GENERIC_NAME,FENTANYL,SCHEDULE II,0,CN101,OPIOID ANALGESICS
2,COCA,COCA,GENERIC_NAME,COCA,SCHEDULE II,2337,CN101,OPIOID ANALGESICS
4,LAUDANUM,LAUDANUM,GENERIC_NAME,LAUDANUM,SCHEDULE II,2338,CN101,OPIOID ANALGESICS
9,SODIUM THIOPENTAL,SODIUM THIOPENTAL,GENERIC_NAME,SODIUM THIOPENTAL,SCHEDULE III,2339,CN101,OPIOID ANALGESICS
27,CATHINE,CATHINE,GENERIC_NAME,CATHINE,SCHEDULE IV,2340,CN101,OPIOID ANALGESICS
...,...,...,...,...,...,...,...,...
2298,FLUOXETINE,FLUOXETINE,GENERIC_NAME,FLUOXETINE,RX-ONLY,2298,CN609,"ANTIDEPRESSANTS,OTHER"
2304,SERTRALINE,SERTRALINE,GENERIC_NAME,SERTRALINE,RX-ONLY,2304,CN609,"ANTIDEPRESSANTS,OTHER"
2310,PHENTERMINE,PHENTERMINE,GENERIC_NAME,PHENTERMINE,SCHEDULE IV,2310,GA751,CENTRALLY-ACTING APPETITE SUPPRESSANTS
2317,NANDROLONE,NANDROLONE,GENERIC_NAME,NANDROLONE,SCHEDULE III,2317,HS100,ANDROGENS/ANABOLICS


In [42]:
joined_drugs_df = joined_drugs_df.rename(columns={'r_VA_CLASS': 'VA_CLASS', 
                                            'r_VA Category': 'VA_CATEGORY'})
joined_drugs_df = joined_drugs_df.drop(['id'], axis=1)

In [43]:
joined_drugs_df

,NAME,GENERIC_NAME,NAME_TYPE,FULL_GENERIC_NAME,FDA_SCHEDULE,VA_CLASS,VA_CATEGORY
0,FENTANYL,FENTANYL,GENERIC_NAME,FENTANYL,SCHEDULE II,CN101,OPIOID ANALGESICS
1,APACHE,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II,CN101,OPIOID ANALGESICS
2,BIRRIA,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II,CN101,OPIOID ANALGESICS
2,COCA,COCA,GENERIC_NAME,COCA,SCHEDULE II,CN101,OPIOID ANALGESICS
3,BLONDE,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II,CN101,OPIOID ANALGESICS
...,...,...,...,...,...,...,...
2332,PUMPERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III,HS900,"HORMONES/SYNTHETICS/MODIFIERS,OTHER"
2333,ROIDS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III,HS900,"HORMONES/SYNTHETICS/MODIFIERS,OTHER"
2334,STACKERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III,HS900,"HORMONES/SYNTHETICS/MODIFIERS,OTHER"
2335,WEIGHT GAINERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III,HS900,"HORMONES/SYNTHETICS/MODIFIERS,OTHER"


In [44]:
#joined_drugs_df.to_csv('./data/cleaned_files/drug_names_fda_VA.csv', index=False)

# Fill missing VA categories

In [45]:
# Pull hypernym from scheduled drugs
# Note: all hypernyms are not very descriptive, however some are better than nothing
query = '''
    SELECT ?drug_name, ?hypernym
    WHERE {
         ?drug dbp:legalUs ?sched .
         ?drug rdfs:label ?drug_name
         OPTIONAL {
             ?drug <http://purl.org/linguistics/gold/hypernym> ?hyper .
             ?hyper rdfs:label ?hypernym
         }
         FILTER (LANG(?drug_name) = "en")
         FILTER (LANG(?hypernym) = "en")
}
 '''
r = requests.get(url, params={'format': 'json', 'query': query})
data = r.json()

In [46]:
data

{'head': {'link': [], 'vars': ['drug_name', 'hypernym']},
 'results': {'distinct': False,
  'ordered': True,
  'bindings': [{'drug_name': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Dryvax'},
    'hypernym': {'type': 'literal', 'xml:lang': 'en', 'value': 'Vaccine'}},
   {'drug_name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Cervarix'},
    'hypernym': {'type': 'literal', 'xml:lang': 'en', 'value': 'Vaccine'}},
   {'drug_name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Gardasil'},
    'hypernym': {'type': 'literal', 'xml:lang': 'en', 'value': 'Vaccine'}},
   {'drug_name': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Zolmitriptan'},
    'hypernym': {'type': 'literal', 'xml:lang': 'en', 'value': 'Agonist'}},
   {'drug_name': {'type': 'literal',
     'xml:lang': 'en',
     'value': 'Bromocriptine'},
    'hypernym': {'type': 'literal', 'xml:lang': 'en', 'value': 'Agonist'}},
   {'drug_name': {'type': 'literal', 'xml:lang': 'en', 'value': 'Liraglutide'

In [47]:
results = {}
for item in data['results']['bindings']:
    drug_name = item['drug_name']['value'].upper()
    hypernym = item['hypernym']['value'].upper()
    if drug_name not in results:
        results[drug_name] = hypernym

In [48]:
results

{'DRYVAX': 'VACCINE',
 'CERVARIX': 'VACCINE',
 'GARDASIL': 'VACCINE',
 'ZOLMITRIPTAN': 'AGONIST',
 'BROMOCRIPTINE': 'AGONIST',
 'LIRAGLUTIDE': 'AGONIST',
 'LINACLOTIDE': 'AGONIST',
 'ARFORMOTEROL': 'AGONIST',
 'INDACATEROL': 'AGONIST',
 'PALIPERIDONE': 'ANTAGONIST',
 'ACLIDINIUM BROMIDE': 'ANTAGONIST',
 'RANIBIZUMAB': 'FRAGMENT',
 'ADAPALENE': 'RETINOID',
 'SODIUM THIOPENTAL': 'ANALOGUE',
 'MEPRODINE': 'ANALOGUE',
 'UNOPROSTONE': 'ANALOGUE',
 'CARFENTANIL': 'ANALOGUE',
 'HYDROMORPHINOL': 'ANALOGUE',
 'TOCERANIB': 'INHIBITOR',
 '1,4,6-ANDROSTATRIENE-3,17-DIONE': 'INHIBITOR',
 'PERINDOPRIL': 'INHIBITOR',
 'EMTRICITABINE': 'INHIBITOR',
 'ZALCITABINE': 'INHIBITOR',
 'DEXLANSOPRAZOLE': 'INHIBITOR',
 'NICOTINE': 'ALKALOID',
 'CATHINONE': 'ALKALOID',
 'PHYSOSTIGMINE': 'ALKALOID',
 'YOHIMBINE': 'ALKALOID',
 'CLENBUTEROL': 'AMINE',
 'AVIBACTAM': 'BEING',
 'EMTRICITABINE/RILPIVIRINE/TENOFOVIR': 'COMBINATION',
 'POMALIDOMIDE': 'DERIVATIVE',
 'BUPRENORPHINE': 'DERIVATIVE',
 'DESOMORPHINE': 'DERIVA

In [49]:
joined_drugs_df[joined_drugs_df['VA_CATEGORY'].isna()].drop_duplicates(subset=['GENERIC_NAME'])

,NAME,GENERIC_NAME,NAME_TYPE,FULL_GENERIC_NAME,FDA_SCHEDULE,VA_CLASS,VA_CATEGORY
52,HEROIN,HEROIN,GENERIC_NAME,HEROIN,SCHEDULE I,NaN,NaN
52,HETEROCODEINE,HETEROCODEINE,GENERIC_NAME,HETEROCODEINE,SCHEDULE II,NaN,NaN
53,MEPRODINE,MEPRODINE,GENERIC_NAME,MEPRODINE,SCHEDULE I,NaN,NaN
54,METAZOCINE,METAZOCINE,GENERIC_NAME,METAZOCINE,SCHEDULE II,NaN,NaN
56,PHENOMORPHAN,PHENOMORPHAN,GENERIC_NAME,PHENOMORPHAN,SCHEDULE I,NaN,NaN
...,...,...,...,...,...,...,...
2165,PHENCYCLIDINE,PCP,BRAND_NAME,PCP,NaN,NaN,NaN
2178,PEYOTE,PEYOTE,GENERIC_NAME,PEYOTE,NaN,NaN,NaN
2186,SALVIA,SALVIA,GENERIC_NAME,SALVIA,NaN,NaN,NaN
2234,DXM,DXM,GENERIC_NAME,DXM,NaN,NaN,NaN


In [50]:
# Function to fill nan VA Categories with hypernym
def fill_na_cat(x, cat_dict):
    generic_name = x['GENERIC_NAME']
    cat = x['VA_CATEGORY']
    if str(cat) == 'nan' and generic_name in cat_dict:
        cat = cat_dict[generic_name]
    
    return cat

In [51]:
joined_drugs_df['CATEGORY'] = joined_drugs_df.apply(lambda x: fill_na_cat(x, results), axis=1)

In [52]:
joined_drugs_df

,NAME,GENERIC_NAME,NAME_TYPE,FULL_GENERIC_NAME,FDA_SCHEDULE,VA_CLASS,VA_CATEGORY,CATEGORY
0,FENTANYL,FENTANYL,GENERIC_NAME,FENTANYL,SCHEDULE II,CN101,OPIOID ANALGESICS,OPIOID ANALGESICS
1,APACHE,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II,CN101,OPIOID ANALGESICS,OPIOID ANALGESICS
2,BIRRIA,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II,CN101,OPIOID ANALGESICS,OPIOID ANALGESICS
2,COCA,COCA,GENERIC_NAME,COCA,SCHEDULE II,CN101,OPIOID ANALGESICS,OPIOID ANALGESICS
3,BLONDE,FENTANYL,STREET_NAME,FENTANYL,SCHEDULE II,CN101,OPIOID ANALGESICS,OPIOID ANALGESICS
...,...,...,...,...,...,...,...,...
2332,PUMPERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III,HS900,"HORMONES/SYNTHETICS/MODIFIERS,OTHER","HORMONES/SYNTHETICS/MODIFIERS,OTHER"
2333,ROIDS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III,HS900,"HORMONES/SYNTHETICS/MODIFIERS,OTHER","HORMONES/SYNTHETICS/MODIFIERS,OTHER"
2334,STACKERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III,HS900,"HORMONES/SYNTHETICS/MODIFIERS,OTHER","HORMONES/SYNTHETICS/MODIFIERS,OTHER"
2335,WEIGHT GAINERS,TESTOSTERONE,STREET_NAME,TESTOSTERONE,SCHEDULE III,HS900,"HORMONES/SYNTHETICS/MODIFIERS,OTHER","HORMONES/SYNTHETICS/MODIFIERS,OTHER"


In [53]:
#joined_drugs_df.to_csv('./data/cleaned_files/drug_names_fda_VA_filled.csv', index=False)